In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
display(p)

train = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train.csv")
train_label = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train_label.csv")
test = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\test.csv")

train_null = train.isnull()

for i in range(train.shape[0]):
    if train_null["company"][i] == False and train_null["agent"][i] == True:
        train["agent"][i] = train["company"][i]

train_filled_value = train.fillna({"children": 0, "country": "PRT","agent":9.0})

test_null = test.isnull()

for i in range(test.shape[0]):
    if test_null["company"][i] == False and test_null["agent"][i] == True:
        test["agent"][i] = test["company"][i]

test_filled_value = test.fillna({"country": "PRT","agent":9.0})

IntProgress(value=0)

<ipython-input-39-46e996aa58ac>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["agent"][i] = train["company"][i]
<ipython-input-39-46e996aa58ac>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["agent"][i] = test["company"][i]


In [40]:
train_filled_value = train_filled_value.drop(["ID","hotel","is_canceled","company","adr","reservation_status","reservation_status_date"], axis=1)
test_filled_value = test_filled_value.drop(["ID","hotel","company"], axis=1)

df_alldata = pd.concat([train_filled_value, test_filled_value], axis=0)

In [74]:
category_column=["ID","hotel","is_canceled","arrival_date_year","arrival_date_month","meal","country","market_segment",
                 "distribution_channel","is_repeated_guest","reserved_room_type","assigned_room_type","deposit_type",
                "agent","customer_type","reservation_status","reservation_status_date"]
numerical_column=["lead_time","arrival_date_week_number","arrival_date_day_of_month","stays_in_weekend_nights",
                 "stays_in_week_nights","adults","children","babies","previous_cancellations","previous_bookings_not_canceled",
                 "booking_changes","days_in_waiting_list","required_car_parking_spaces","total_of_special_requests"]

category_column_new=[]
numerical_column_new=[]
for i in df_alldata:
    if i in category_column:
        category_column_new.append(i)
    if i in numerical_column:
        numerical_column_new.append(i)
for j in category_column_new:
    df_alldata[j] = df_alldata[j].astype('str')
df_numerical = df_alldata.loc[:,numerical_column_new]
df_category = df_alldata.loc[:,category_column_new]
df_category = pd.get_dummies(df_category)
df2 = pd.concat( [df_category, df_numerical], axis=1 )

df_train=df2[:91531]
df_test=df2[91531:]
#df_test.info()

df_train_numerical=df_numerical[:91531]
df_test_numerical=df_numerical[91531:]
df_train_category=df_category[:91531]
df_test_category=df_category[91531:]

In [6]:
from xgboost.sklearn import XGBClassifier
x = df_train
y = train["is_canceled"]
y = np.array(y,dtype=int)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
#XGboost
xgclf=XGBClassifier(n_estimators=100,learning_rate= 0.2,max_depth=6,subsample=1,gamma=0,reg_lambda=1,max_delta_step=0,objective='binary:logistic',colsample_bytree=1)
xgclf.fit(X_train,y_train)
y_iscanceled_predict_train = xgclf.predict(X_train)
y_iscanceled_predict_test = xgclf.predict(X_test)
score_xg = xgclf.score(X_test,y_test)
print("XGBoost Accuracy = ",score_xg*100," %")
y_iscanceled_predict = np.concatenate((y_iscanceled_predict_train,y_iscanceled_predict_test),axis=0)

[23:02:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Accuracy =  88.37603102638334  %


In [7]:
from sklearn.model_selection import KFold
x = df_train
x = np.array(x)
y = train["is_canceled"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    xgclf=XGBClassifier(n_estimators=100,learning_rate= 0.2,max_depth=6,subsample=1,gamma=0,reg_lambda=1,max_delta_step=0,objective='binary:logistic',colsample_bytree=1)
    xgclf.fit(X_train,y_train)
    y_iscanceled_predict = xgclf.predict(X_train)
    accuracy = xgclf.score(X_test,y_test)
    print("XGBoost Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_xgclf = xgclf
print("Best Accuracy = ",best_accuracy*100,"%")

[23:08:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Accuracy =  86.28939749822472 %
[23:10:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Accuracy =  88.80148585163334 %
[23:12:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old 

In [8]:
#final accuracy
x_train_new = x
y_train_new = train["is_canceled"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred_is_canceled = best_xgclf.predict(x_train_new)
count = 0

for i in range(y_pred_is_canceled.shape[0]):
    if y_pred_is_canceled[i] != y_train_new[i]:
        count += 1
xg_accuracy = 1-count/y_pred_is_canceled.shape[0]
print("XGBoost Accuracy = ",xg_accuracy*100,"%")

XGBoost Accuracy =  89.06599949743803 %


In [9]:
#XGboost adr
from xgboost.sklearn import XGBRegressor
x = df_train
y = train["adr"]
y = np.array(y)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
xge = XGBRegressor(learning_rate=0.1,n_estimators=200,
                                     max_depth=6, min_child_weight=0,
                                     gamma=0, subsample=1,
                                     colsample_bytree=1,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
xge.fit(X_train,y_train)
y_adr_predict_train = xge.predict(X_train)
y_adr_predict_test = xge.predict(X_test)
a=xge.score(X_test,y_test)
mse_xge = np.mean((y_adr_predict_train - y_train) ** 2)
print("MSE = ",mse_xge)
print(y_adr_predict_train)
print(y_adr_predict_test)
y_adr_predict = np.concatenate((y_adr_predict_train,y_adr_predict_test),axis=0)
print(y_adr_predict)

[23:24:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
MSE =  472.297421419346
[140.17241  76.85319  74.18597 ... 143.70161 118.34753  43.02816]
[ 76.48157   61.362865  42.468266 ...  37.764893 131.20723   53.726   ]
[140.17241   76.85319   74.18597  ...  37.764893 131.20723   53.726   ]


In [58]:
from sklearn.model_selection import KFold
x = df_train
x = np.array(x)
y = train["adr"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    xge = XGBRegressor(learning_rate=0.1,n_estimators=200,
                                     max_depth=6, min_child_weight=0,
                                     gamma=0, subsample=1,
                                     colsample_bytree=1,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)
    xge.fit(X_train,y_train)
    y_adr_predict = xge.predict(X_train)
    accuracy = xge.score(X_test,y_test)
    print("XGBoost Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_xge = xge
print("Best Accuracy = ",best_accuracy*100,"%")

[00:53:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Accuracy =  47.1137772918181 %
[00:57:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Accuracy =  21.527997622493146 %
[01:00:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Accuracy =  65.13196214076945 %
[01:04:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
XGBoost Accuracy =  69.47814524662613 %
[01:08:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/objective/regression_obj.cu:170: reg:linear is 

In [62]:
x_train_new = x
y_train_new = train["adr"]
y_train_new = y_train_new.reset_index(drop=True)

y_adr_predict = best_xge.predict(x_train_new)
count = 0

for i in range(y_adr_predict.shape[0]):
    if y_adr_predict[i] != y_train_new[i]:
        count += 1
xge_accuracy = 1-count/y_adr_predict.shape[0]
print("XGBoost Accuracy = ",xge_accuracy*100,"%")

XGBoost Accuracy =  0.0 %


In [64]:
traincolumn = ["is_canceled","arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights","adr"]
df_train = train[traincolumn]
df_train["is_canceled_predict"] = y_pred_is_canceled
df_train["adr_predict"] = y_adr_predict

date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_train.shape[0]):
    data = f"{df_train['arrival_date_year'][i]}/{month_dict[df_train['arrival_date_month'][i]]}/{df_train['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)
df_train["arrival_date"] = np.array(date_list)
df_train = df_train.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_train.head()

<ipython-input-64-53cc57a97f91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["is_canceled_predict"] = y_pred_is_canceled
<ipython-input-64-53cc57a97f91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["adr_predict"] = y_adr_predict
<ipython-input-64-53cc57a97f91>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adr,is_canceled_predict,adr_predict,arrival_date
0,0,0,0,-6.305161,0,-12.928834,2015/07/01
1,0,0,2,75.052227,0,49.955837,2015/07/01
2,0,0,2,74.546401,0,75.078186,2015/07/01
3,0,0,2,76.376288,0,75.078186,2015/07/01
4,0,0,2,49.411647,0,75.078186,2015/07/01


In [65]:
adr_list = []
is_canceled_list = []
for i in range(df_train.shape[0]):
    adr_list.append(abs(df_train["adr"][i] - df_train["adr_predict"][i]))
    is_canceled_list.append(abs(df_train["is_canceled"][i] - df_train["is_canceled_predict"][i]))
print("adr Ein = ",sum(adr_list)/df_train.shape[0])
print("is_canceled Ein = ",sum(is_canceled_list)/df_train.shape[0])

adr Ein =  16.34230751010642
is_canceled Ein =  0.10934000502561973


In [66]:
stay_list = []
for i in range(df_train.shape[0]):
    stay = df_train["stays_in_weekend_nights"][i] + df_train["stays_in_week_nights"][i]
    stay_list.append(stay)

df_train["stay"] = np.array(stay_list)
df_train = df_train.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

df_train = df_train.drop(["adr","is_canceled"],axis=1)

In [67]:
condition = df_train["is_canceled_predict"] == 0
df_train = df_train[condition]
df_train = df_train.reset_index(drop=True)

In [68]:
daily_revenue_list = []
for i in range(df_train.shape[0]):
    daily_revenue = df_train["adr_predict"][i] * df_train["stay"][i]
    daily_revenue_list.append(daily_revenue) 

df_train["daily_revenue"] = np.array(daily_revenue_list)
df_train = df_train.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [69]:
drop = []
head = 0
for i in range(1,df_train.shape[0]):
    if df_train["arrival_date"][i] == df_train["arrival_date"][head]: 
        df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit = df_train.drop(drop)
submit = submit.reset_index(drop=True)
submit

<ipython-input-69-8081819b65db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]


,arrival_date,daily_revenue
0,2015/07/01,22609.663601
1,2015/07/02,17561.411556
2,2015/07/03,14984.400999
3,2015/07/04,18009.977175
4,2015/07/05,20284.411957
...,...,...
635,2017/03/27,27755.800466
636,2017/03/28,15244.034149
637,2017/03/29,24229.413687
638,2017/03/30,30786.371151


In [70]:
submit["label"] = train_label["label"]
predict_list = []
Ein = 0
for i in range(submit.shape[0]):
    predict = submit["daily_revenue"][i]//10000
    predict_list.append(predict)
    #if submit["label"][i] != submit["predict"][i]:
        #Ein+=1
submit["predict"] = np.array(predict_list)
submit

,arrival_date,daily_revenue,label,predict
0,2015/07/01,22609.663601,2.0,2.0
1,2015/07/02,17561.411556,1.0,1.0
2,2015/07/03,14984.400999,1.0,1.0
3,2015/07/04,18009.977175,1.0,1.0
4,2015/07/05,20284.411957,1.0,2.0
...,...,...,...,...
635,2017/03/27,27755.800466,2.0,2.0
636,2017/03/28,15244.034149,1.0,1.0
637,2017/03/29,24229.413687,2.0,2.0
638,2017/03/30,30786.371151,3.0,3.0


In [71]:
Ein = 0
for i in range(submit.shape[0]):
    if submit["label"][i] != submit["predict"][i]:
        Ein+=1
print("Ein = ",Ein/submit.shape[0])

Ein =  0.21875


In [75]:
train_x = df_train.values
test_x = df_test.values
y = train["is_canceled"]
best_xgclf.fit(train_x,y)
y_test_iscanceled_predict = best_xgclf.predict(test_x)
y_test_adr_predict = best_xge.predict(test_x)
testcolumn = ["arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights"]
df_test = test[testcolumn]
df_test["is_canceled_predict"] = y_test_iscanceled_predict
df_test["adr_predict"] = y_test_adr_predict
df_test.head()

[01:17:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


<ipython-input-75-096768156a2a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["is_canceled_predict"] = y_test_iscanceled_predict
<ipython-input-75-096768156a2a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["adr_predict"] = y_test_adr_predict


,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,is_canceled_predict,adr_predict
0,2017,April,1,2,5,0,93.287567
1,2017,April,1,4,10,1,24.161552
2,2017,April,1,2,5,0,57.323982
3,2017,April,1,2,5,1,121.058075
4,2017,April,1,2,4,1,188.542953


In [76]:
date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_test.shape[0]):
    data = f"{df_test['arrival_date_year'][i]}-{month_dict[df_test['arrival_date_month'][i]]}-{df_test['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)

df_test["arrival_date"] = np.array(date_list)
df_test = df_test.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27859 entries, 0 to 27858
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   stays_in_weekend_nights  27859 non-null  int64  
 1   stays_in_week_nights     27859 non-null  int64  
 2   is_canceled_predict      27859 non-null  int64  
 3   adr_predict              27859 non-null  float32
 4   arrival_date             27859 non-null  object 
dtypes: float32(1), int64(3), object(1)
memory usage: 979.5+ KB


<ipython-input-76-762c1d70825e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["arrival_date"] = np.array(date_list)


In [77]:
stay_list = []
for i in range(df_test.shape[0]):
    stay = df_test["stays_in_weekend_nights"][i] + df_test["stays_in_week_nights"][i]
    stay_list.append(stay)

df_test["stay"] = np.array(stay_list)
df_test = df_test.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

In [78]:
condition = df_test["is_canceled_predict"] == 0
df_test = df_test[condition]
df_test = df_test.reset_index(drop=True)

In [79]:
daily_revenue_list = []
for i in range(df_test.shape[0]):
    daily_revenue = df_test["adr_predict"][i] * df_test["stay"][i]
    daily_revenue_list.append(daily_revenue)

In [80]:
df_test["daily_revenue"] = np.array(daily_revenue_list)
df_test = df_test.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [81]:
drop = []
head = 0
for i in range(1,df_test.shape[0]):
    if df_test["arrival_date"][i] == df_test["arrival_date"][head]: 
        df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit_test = df_test.drop(drop)
submit_test = submit_test.reset_index(drop=True)
submit_test

<ipython-input-81-29dd57138732>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]


,arrival_date,daily_revenue
0,2017-04-01,26675.709827
1,2017-04-02,27246.864214
2,2017-04-03,39247.179352
3,2017-04-04,13717.733446
4,2017-04-05,34238.498579
...,...,...
148,2017-08-27,54223.477234
149,2017-08-28,70163.631557
150,2017-08-29,36714.956539
151,2017-08-30,32021.804802


In [82]:
predict_list = []
for i in range(submit_test.shape[0]):
    predict = submit_test["daily_revenue"][i]//10000
    predict_list.append(predict)
submit_test["predict"] = np.array(predict_list)

In [83]:
nolabel = pd.read_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv")
nolabel

,arrival_date,label
0,4/1/2017,2.0
1,4/2/2017,2.0
2,4/3/2017,3.0
3,4/4/2017,1.0
4,4/5/2017,3.0
...,...,...
148,8/27/2017,5.0
149,8/28/2017,7.0
150,8/29/2017,3.0
151,8/30/2017,3.0


In [84]:
nolabel["label"] = np.array(predict_list)
nolabel

,arrival_date,label
0,4/1/2017,2.0
1,4/2/2017,2.0
2,4/3/2017,3.0
3,4/4/2017,1.0
4,4/5/2017,3.0
...,...,...
148,8/27/2017,5.0
149,8/28/2017,7.0
150,8/29/2017,3.0
151,8/30/2017,3.0


In [85]:
nolabel.to_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv",index=False)
nolabel["label"].value_counts()

3.0    45
4.0    34
2.0    27
5.0    22
6.0    15
7.0     5
1.0     3
9.0     1
8.0     1
Name: label, dtype: int64